<a href="https://colab.research.google.com/github/kooxc/Jupyter-Notebook/blob/main/langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 一、安装

In [ ]:
# 查看当前kernel下已安装的包  list packages
!pip list --format=columns

In [ ]:
# 安装 langchain 包
!pip install langchain
!pip install langchain-core
!pip install langchain-community
!pip install langchain-openai

In [ ]:
# 安装阿里通义千问的包
!pip install dashscope

### 二、基本调用示例

#### 1、使用ChatTongyi访问

In [24]:
import os
from langchain_community.llms import ChatTongyi
from google.colab import userdata

os.environ["DASHSCOPE_API_KEY"] = userdata.get('QWEN_KEY')

llm = ChatTongyi(model="qwen-turbo")
llm.invoke("50个字介绍一下明源云这家公司?")

'明源云是一家专注于不动产领域的数字化解决方案提供商，致力于为房地产企业及上下游生态链提供全生命周期的数字化产品与服务，助力行业转型升级，提升运营效率与客户体验。'

#### 2、使用ChatOpenAI访问

In [34]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(
    model="qwen-turbo",
    base_url=userdata.get('QWEN_BASE'),
    openai_api_key= userdata.get('QWEN_KEY')
)
llm.invoke("50个字介绍一下明源云这家公司?")

ImportError: cannot import name 'convert_to_openai_image_block' from 'langchain_core.messages' (/usr/local/lib/python3.11/dist-packages/langchain_core/messages/__init__.py)

### 三、聊天模型

In [25]:
from langchain_community.chat_models.tongyi import ChatTongyi
llm = ChatTongyi(model="qwen-turbo")

#### 1、格式化输出

In [ ]:

from typing import Optional
from pydantic import BaseModel, Field

# 定义Pydantic模型
class Company(BaseModel):
    """公司简介信息"""

    domain: str = Field(description="官方域名地址")
    business: str = Field(description="主营业务")
    boss: str = Field(description="老板")
    income: str = Field(description="最近市场披露的收入")


structured_llm = llm.with_structured_output(Company)
structured_llm.invoke("介绍一下明源云这家公司")

Company(domain='www.mingyuanyun.com', business='房地产行业数字化转型解决方案提供商', boss='高宇', income='2022年收入超过10亿元')

#### 2、工具模式

In [ ]:
# 函数名、类型和文档注释都是该工具的一部分
# 传递给模型的模式，定义良好的描述性模式，是提示工程的扩展，是使模型性能良好的重要组成部分
def add(a: int, b: int) -> int:
    """Add two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a + b


def multiply(a: int, b: int) -> int:
    """Multiply two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a * b

tools = [add, multiply]

In [ ]:
llm_with_tools = llm.bind_tools(tools)

query = "3 * 12 = ？"

llm_with_tools.invoke(query)

#### 3、缓存
在 LangChain 中，set_llm_cache 是一个用于设置缓存机制的函数。它的主要作用是通过缓存来减少对语言模型（LLM）的重复调用，从而节省时间和计算资源。以下是它的具体作用和使用场景：

**缓存的作用**

减少重复调用：如果相同的输入多次发送到 LLM，缓存可以存储之前的结果，避免重复调用模型，从而提高效率。
节省成本和时间：LLM 的调用通常需要消耗计算资源和时间，缓存可以显著减少这些开销。
提高性能：通过缓存，程序可以更快地获取结果，尤其是在需要频繁调用 LLM 的场景中。

**如何使用 set_llm_cache**

set_llm_cache 是 LangChain 提供的一个全局设置函数，用于启用缓存机制。它通常与 LangChain 的缓存实现（如 InMemoryCache 或 SQLiteCache）一起使用。

In [ ]:
%%time
from langchain_core.caches import InMemoryCache
from langchain_core.globals import set_llm_cache


set_llm_cache(InMemoryCache())
llm.invoke("50个字以内介绍一下明源云这家公司")

In [ ]:
%%time
#再次调用

llm.invoke("50个字以内介绍一下明源云这家公司")

#### 4、对话消息
- SystemMessage: 描述问题的背景,或者描述大模型的角色
- HumanMessage: 用户输入的问题
- AIMessage: 大模型输入的答案，另外还有一种消息是function_call

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage,AIMessage

message=[
    SystemMessage("你想了解哪个公司信息，我可以在50个字以内介绍这家公司"),
    HumanMessage("明源云")
]
llm.invoke(message)

#其他多种调用方式：
# llm.invoke([{'role':'user','content':'用友'}])

其他多种调用方式：

In [ ]:
llm.invoke([
    {'role':'system','content':'你想了解哪个公司信息，我可以在50个字以内介绍这家公司'},
     {'role':'user','content':'用友'}
])